In [0]:
import os
import time

# Reference

* https://github.com/Kajiyu/dnc-py3.git
* https://github.com/Mostafa-Samir/DNC-tensorflow.git

# Source Code

In [0]:
!git clone https://github.com/Kajiyu/dnc-py3.git
!git clone https://github.com/Mostafa-Samir/DNC-tensorflow.git

Cloning into 'dnc-py3'...
remote: Enumerating objects: 317, done.
remote: Total 317 (delta 0), reused 0 (delta 0), pack-reused 317
Receiving objects: 100% (317/317), 43.27 MiB | 23.24 MiB/s, done.
Resolving deltas: 100% (197/197), done.
Checking out files: 100% (115/115), done.
Cloning into 'DNC-tensorflow'...
remote: Enumerating objects: 641, done.
remote: Total 641 (delta 0), reused 0 (delta 0), pack-reused 641
Receiving objects: 100% (641/641), 89.78 MiB | 28.12 MiB/s, done.
Resolving deltas: 100% (317/317), done.
Checking out files: 100% (56/56), done.


In [0]:
!mv ./DNC-tensorflow/tasks/babi/checkpoints ./dnc-py3/tasks/babi/

In [0]:
print(next(os.walk("./dnc-py3/tasks/babi/checkpoints"))[1])

['step-500005']


# bABI

In [0]:
import os
import sys

sys.path.append(os.path.join(".","dnc-py3"))
from dnc.dnc import DNC

sys.path.append(os.path.join(".","dnc-py3","tasks","babi"))
from recurrent_controller import RecurrentController

import tensorflow as tf
import numpy as np
import pickle
import re

In [0]:
def llprint(message):
    sys.stdout.write(message)
    sys.stdout.flush()

In [0]:
def load(path):
    return pickle.load(open(path, 'rb'))

In [0]:
def onehot(index, size):
    vec = np.zeros(size, dtype=np.float32)
    vec[int(index)] = 1.0
    return vec

In [0]:
def prepare_sample(sample, target_code, word_space_size):
    """
    Input:
      sample: train or test data with np.array([{'inputs':[],'outputs':[]}])
      target_code: the end word in the inputs, here is '-'
      word_space_size: the word space, here is 159
    Output:
      [0]: input_vec with one-hot encoding as [batch_size=1, word_count, word_space_size]
      [1]: output_vec with one-hot encoding as the same with input_vec
           but replacing target_code with sample["outputs"]
      [2]: seq_len: word_count
      [3]: weights as [batch_size=1, word_count, 1] to mark target_code as 1.0,
           the other is zero
    """
    input_vec = np.array(sample[0]['inputs'], dtype=np.float32)
    output_vec = np.array(sample[0]['inputs'], dtype=np.float32)
    seq_len = input_vec.shape[0]
    weights_vec = np.zeros(seq_len, dtype=np.float32)

    target_mask = (input_vec == target_code)
    output_vec[target_mask] = sample[0]['outputs']
    weights_vec[target_mask] = 1.0

    input_vec = np.array([onehot(int(code), int(word_space_size)) for code in input_vec])
    output_vec = np.array([onehot(int(code), int(word_space_size)) for code in output_vec])

    return (
        np.reshape(input_vec, (1, -1, word_space_size)),
        np.reshape(output_vec, (1, -1, word_space_size)),
        seq_len,
        np.reshape(weights_vec, (1, -1, 1))
    )

In [0]:
prj_path = os.path.join(".","dnc-py3","tasks","babi")
ckpts_dir = os.path.join(".","dnc-py3","tasks","babi","checkpoints")
lexicon_dictionary = load(os.path.join(prj_path,"data","en-10k","lexicon-dict.pkl"))
question_code = lexicon_dictionary["?"]
target_code = lexicon_dictionary["-"]
test_files = []

for entryname in os.listdir(os.path.join(prj_path,"data","en-10k","test")):
    entry_path = os.path.join(os.path.join(prj_path,"data","en-10k","test", entryname))
    if os.path.isfile(entry_path):
        test_files.append(entry_path)

batch_size = 1        

## Starting Training

In [0]:
dirname = os.path.join(".","dnc-py3","tasks","babi")
ckpts_dir = os.path.join(dirname , 'checkpoints')
data_dir = os.path.join(dirname, 'data', 'en-10k')
tb_logs_dir = os.path.join(dirname, 'logs')

llprint("Loading Data ... ")
lexicon_dict = load(os.path.join(data_dir, 'lexicon-dict.pkl'))
data = load(os.path.join(data_dir, 'train', 'train.pkl'))
llprint("Done!\n")

batch_size = 1
input_size = output_size = len(lexicon_dict)
sequence_max_length = 100
word_space_size = len(lexicon_dict)
words_count = 256
word_size = 64
read_heads = 4

learning_rate = 1e-4
momentum = 0.9

from_checkpoint = None
iterations = 100000
start_step = 0

Loading Data ... Done!


In [0]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session(graph=graph) as session:

        llprint("Building Computational Graph ... ")

        optimizer = tf.train.RMSPropOptimizer(learning_rate, momentum=momentum)
        summerizer = tf.summary.FileWriter(tb_logs_dir, session.graph)

        ncomputer = DNC(
            RecurrentController,
            input_size,
            output_size,
            sequence_max_length,
            words_count,
            word_size,
            read_heads,
            batch_size
        )

        output, memory_views = ncomputer.get_outputs()

        loss_weights = tf.placeholder(tf.float32, [batch_size, None, 1])
        loss = tf.reduce_mean(
            loss_weights * \
            tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, \
                                                       labels=ncomputer.target_output)
        )

        summeries = []

        gradients = optimizer.compute_gradients(loss)
        for i, (grad, var) in enumerate(gradients):
            if grad is not None:
                gradients[i] = (tf.clip_by_value(grad, -10, 10), var)
        for (grad, var) in gradients:
            if grad is not None:
                summeries.append(tf.summary.histogram(var.name + '/grad', grad))

        apply_gradients = optimizer.apply_gradients(gradients)

        summeries.append(tf.summary.scalar("Loss", loss))

        summerize_op = tf.summary.merge(summeries)
        no_summerize = tf.no_op()

        llprint("Done!\n")

        llprint("Initializing Variables ... ")
        session.run(tf.global_variables_initializer())
        llprint("Done!\n")

        if from_checkpoint is not None:
            llprint("Restoring Checkpoint %s ... " % (from_checkpoint))
            ncomputer.restore(session, ckpts_dir, from_checkpoint)
            llprint("Done!\n")


        last_100_losses = []

        start = 0 if start_step == 0 else start_step + 1
        end = start_step + iterations + 1

        start_time_100 = time.time()
        end_time_100 = None
        avg_100_time = 0.
        avg_counter = 0

        for i in range(start, end + 1):
            try:
                llprint("\rIteration %d/%d" % (i, end))

                sample = np.random.choice(data, 1)
                input_data, target_output, seq_len, weights = \
                  prepare_sample(sample, lexicon_dict['-'], word_space_size)

                summerize = (i % 10 == 0)
                take_checkpoint = (i != 0) and (i % 1000 == 0)

                loss_value, _, summary = session.run([
                    loss,
                    apply_gradients,
                    summerize_op if summerize else no_summerize
                ], feed_dict={
                    ncomputer.input_data: input_data,
                    ncomputer.target_output: target_output,
                    ncomputer.sequence_length: seq_len,
                    ncomputer.input_mode: np.zeros((batch_size, seq_len, output_size)),
                    loss_weights: weights
                })

                last_100_losses.append(loss_value)
                # summerizer.add_summary(summary, i)

                if summerize:
                    llprint("\n\tAvg. Cross-Entropy: %.7f\n" % (np.mean(last_100_losses)))

                    end_time_100 = time.time()
                    elapsed_time = (end_time_100 - start_time_100) / 60
                    avg_counter += 1
                    avg_100_time += (1. / avg_counter) * (elapsed_time - avg_100_time)
                    estimated_time = (avg_100_time * ((end - i) / 100.)) / 60.

                    print("\tAvg. 100 iterations time: %.2f minutes" % (avg_100_time))
                    print("\tApprox. time to completion: %.2f hours" % (estimated_time))

                    start_time_100 = time.time()
                    last_100_losses = []

                if take_checkpoint:
                    llprint("\nSaving Checkpoint ... "),
                    ncomputer.save(session, ckpts_dir, 'step-%d' % (i))
                    llprint("Done!\n")

            except KeyboardInterrupt:

                llprint("\nSaving Checkpoint ... "),
                ncomputer.save(session, ckpts_dir, 'step-%d' % (i))
                llprint("Done!\n")
                sys.exit(0)


Building Computational Graph ... WARNING:tensorflow:From ./dnc-py3/tasks/babi/recurrent_controller.py:14: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
INFO:tensorflow:Summary name shape_inference/basic_lstm_cell/kernel:0/grad is illegal; using shape_inference/basic_lstm_cell/kernel_0/grad instead.
INFO:tensorflow:Summary name shape_inference/basic_lstm_cell/bias:0/grad is illegal; using shape_inference/basic_lstm_cell/bias_0/grad instead.
INFO:tensorflow:Summary name controller/interface_weights:0/grad is illegal; using controller/interface_weights_0/grad instead.
INFO:tensorflow:Summary name controller/nn_output_weights:0/grad is illegal; using controller/nn_output_weights_0/grad in

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Output Training Weights

In [0]:
print(next(os.walk(ckpts_dir))[1])

['step-27000', 'step-18000', 'step-3000', 'step-4000', 'step-19000', 'step-10000', 'step-12000', 'step-20000', 'step-1000', 'step-33000', 'step-21000', 'step-5000', 'step-32000', 'step-17000', 'step-9000', 'step-22000', 'step-34000', 'step-24000', 'step-29000', 'step-6000', 'step-25000', 'step-7000', 'step-14000', 'step-500005', 'step-35000', 'step-23000', 'step-13000', 'step-15000', 'step-28000', 'step-8000', 'step-16000', 'step-26000', 'step-31000', 'step-30000', 'step-11000', 'step-2000', 'step-36000', 'step-36643']


In [0]:
ckpts_dir

'./dnc-py3/tasks/babi/checkpoints'

In [0]:
!zip -r step ./dnc-py3/tasks/babi/checkpoints/*

  adding: dnc-py3/tasks/babi/checkpoints/step-1000/ (stored 0%)
  adding: dnc-py3/tasks/babi/checkpoints/step-1000/model.ckpt.meta (deflated 94%)
  adding: dnc-py3/tasks/babi/checkpoints/step-1000/model.ckpt.index (deflated 29%)
  adding: dnc-py3/tasks/babi/checkpoints/step-1000/checkpoint (deflated 42%)
  adding: dnc-py3/tasks/babi/checkpoints/step-1000/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: dnc-py3/tasks/babi/checkpoints/step-10000/ (stored 0%)
  adding: dnc-py3/tasks/babi/checkpoints/step-10000/model.ckpt.meta (deflated 94%)
  adding: dnc-py3/tasks/babi/checkpoints/step-10000/model.ckpt.index (deflated 29%)
  adding: dnc-py3/tasks/babi/checkpoints/step-10000/checkpoint (deflated 42%)
  adding: dnc-py3/tasks/babi/checkpoints/step-10000/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: dnc-py3/tasks/babi/checkpoints/step-11000/ (stored 0%)
  adding: dnc-py3/tasks/babi/checkpoints/step-11000/model.ckpt.meta (deflated 94%)
  adding: dnc-py3/tasks/babi/checkpoints/st